<a href="https://colab.research.google.com/github/yimingsun12138/IDE_on_Colab/blob/main/Web_IDE_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check Google Colab IP location
Re-connect the Colab server if you do not like the IP location.

In [ ]:
!curl getip.mrdoge.cool

# Global setting

In [ ]:
#@title IDE settings { display-mode: "form" }

#@markdown Choose the IDE and tunnelling method you prefer.

web_IDE = 'Rstudio server' #@param ["Rstudio server", "code server"]
tunnelling = 'localtunnel' #@param ["localtunnel", "ngrok", "customize"]

In [ ]:
#@title User info { display-mode: "form" }

#@markdown Input the Linux user info.

user_name = 'Knight' #@param { type: "string" }
user_password = 'midnight' #@param { type: "string" }

In [ ]:
#@title Colab settings { display-mode: "form" }

#@markdown Choose whether mount Google Drive.

mount_Google_Drive = False #@param { type: "boolean" }

#@markdown Choose whether output detailed running log.

running_verbose = False #@param { type: "boolean" }

#@markdown Choose whether to keep the Colab notebook busy.

keep_busy = False #@param { type: "boolean" }

# Update environment

In [ ]:
%%capture
!apt update
!apt upgrade
!apt autoremove

# Source code

In [ ]:
%%capture
import os
import subprocess
!git clone https://github.com/yimingsun12138/IDE_on_Colab.git

In [ ]:
class Colab_IDE:
  def __init__(self,IDE,tunnelling,name,password,mount_GD):
    self.IDE = IDE
    self.tunnelling = tunnelling
    self.name = name
    self.password = password
    self.mount_GD = mount_GD
    self.web_port = 8787

    # add Linux user
    self.create_user()

    # mount Google Drive
    self.Google_Drive()

  def create_user(self):
    os.system("useradd -m -s /bin/bash" + " " + str(self.name))
    os.system("echo '" + str(self.name) + ":" + str(self.password) + "' | chpasswd")
    os.system("usermod -G sudo" + " " + str(self.name))
  
  def Google_Drive(self):
    if self.mount_GD:
      from google.colab import drive
      drive.mount('/content/drive')

  # install web IDE

  def Install_Rstudio_server(self,web_port=None,verbose=True):

    # check param
    if (web_port is None):
      web_port = self.web_port

    # install
    char_cmd = "bash /content/IDE_on_Colab/web_IDE/Install_Rstudio_server.sh" + " " + str(web_port)
    Running_log = subprocess.run(char_cmd,capture_output=True,shell=True,check=True,universal_newlines=True)
    if verbose:
      print("stdout:\n" + str(Running_log.stdout).replace('\\n','\n'))
      print("stderr:\n" + str(Running_log.stderr).replace('\\n','\n'))

  def Install_code_server(self,web_port=None,web_password=None,verbose=True):

    # check param
    if (web_port is None):
      web_port = self.web_port
    if (web_password is None):
      web_password = self.password

    # install
    os.system("apt install expect")
    char_cmd = "expect /content/IDE_on_Colab/web_IDE/Install_code_server.exp" + " " + str(self.name) + " " + str(self.password) + " " + str(web_port) + " " + str(web_password)
    Running_log = subprocess.run(char_cmd,capture_output=True,shell=True,check=True,universal_newlines=True)
    char_cmd = "expect /content/IDE_on_Colab/web_IDE/Run_code_server.exp" + " " + str(self.name)
    os.system(char_cmd)
    if verbose:
      print("stdout:\n" + str(Running_log.stdout).replace('\\n','\n'))
      print("stderr:\n" + str(Running_log.stderr).replace('\\n','\n'))

  # run tunnelling
  def Run_localtunnel(self,web_port=None):

    # check param
    if (web_port is None):
      web_port = self.web_port

    # install
    char_cmd = "bash /content/IDE_on_Colab/tunnelling/Run_localtunnel.sh" + " " + str(web_port)
    os.system(char_cmd)

  def Run_ngrok(self,web_port=None):

    # check param
    if (web_port is None):
      web_port = self.web_port

    # input token
    ngrok_token = input("Input your ngrok token:")

    # install
    char_cmd = "bash /content/IDE_on_Colab/tunnelling/Run_ngrok.sh" + " " + str(ngrok_token) + " " + str(web_port)
    os.system(char_cmd)

# Install web IDE

In [ ]:
Google_Colab = Colab_IDE(IDE=web_IDE,tunnelling=tunnelling,name=user_name,password=user_password,mount_GD=mount_Google_Drive)

In [ ]:
if (Google_Colab.IDE == 'Rstudio server'):
  Google_Colab.Install_Rstudio_server(verbose=running_verbose)
elif (Google_Colab.IDE == 'code server'):
  Google_Colab.Install_code_server(verbose=running_verbose)

# Tunnelling

In [ ]:
if (Google_Colab.tunnelling == 'localtunnel'):
  Google_Colab.Run_localtunnel()
  !cat /content/IDE_on_Colab/localtunnel.log | grep "your url is:"
elif (Google_Colab.tunnelling == 'ngrok'):
  Google_Colab.Run_ngrok()
  !cat /content/IDE_on_Colab/ngrok.log | grep 'started tunnel'

# Keep Colab notebook busy

In [ ]:
if (Google_Colab.tunnelling == 'customize'):
  print("Use your own tunnelling method below!")
elif keep_busy:
  import numpy as np
  import time
  ii = 1

  while True:
    i = np.random.rand(100,100)
    i = np.dot(i,i)
    i = 'round ' + str(ii)
    print(i)
    ii = ii + 1
    time.sleep(60)